## Collecting Data

In [1]:
import numpy as np
import cv2
import sys
sys.path.insert(0, "/opt/intel/mkl/lib/intel64")
sys.path.insert(0, "/usr/local/lib/python3.6/site-packages")
import dlib
import numpy as np
import logging
import time

def dlibFacelandmarkToNumpy(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
 
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
 
    return coords
 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

DATA_AMOUNT = 500



In [8]:
import os
print(os.listdir('faces/'))

["P'An", 'Bone', 'Thipok', 'Mum', 'Joyce', 'Book', 'chairoj', "P'Ton", 'Chal', 'Nick', 'models', 'Tan', 'Nut', "P'Dear", 'trainingData', 'Gearng', 'Oat']


In [2]:
import pickle
import csv

STORE_LOCATION = 'faces/'

In [10]:

videoInput = cv2.VideoCapture(0)
data = np.ndarray((DATA_AMOUNT,68,2), dtype = float)
data_count = 0

while (data_count < DATA_AMOUNT):
    
    _,frame = videoInput.read()
   
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    rects = detector(gray, 0)
    for rect in rects:
#         print('found')
        shape = predictor(gray, rect)
        shape = dlibFacelandmarkToNumpy(shape)
        x, y, w, h = cv2.boundingRect(shape)
#         print(shape)
#         print('x:',x, 'y:', y, 'w:', w,'h:', h)
        shape[:,0] -= x
        shape[:,1] -= y
        shape_x = shape[:,0]
        shape_y = shape[:,1]
        shape = shape.astype(float)
        shape[:,0] = np.divide(shape_x, w)
        shape[:,1] = np.divide(shape_y, h)
        data[data_count,:,:] = shape
        print(int(data_count * 100 / DATA_AMOUNT),end = '\r')
#         print(shape)
        data_count += 1
# do a bit of cleanup
cv2.destroyAllWindows()
videoInput.release()
# print(data)
name = input("Enter name: ")
pickle.dump(data,open(STORE_LOCATION + name, 'wb'))
print('stored')



error: /home/thananop/Downloads/opencv/opencv-3.4.0/modules/imgproc/src/color.cpp:11111: error: (-215) scn == 3 || scn == 4 in function cvtColor


## Get data

In [3]:
import torch
def unpickle(file):
    with open(file,'rb') as fo:
        output = pickle.load(fo, encoding = 'ASCII')
    return output
n = int(input('amount of people:'))

classes = list()

TRAINING_AMOUNT = int(n * DATA_AMOUNT * 4 / 5)
TEST_AMOUNT = int(n * DATA_AMOUNT * 1 / 5)

training_data = np.ndarray((TRAINING_AMOUNT,68*2))
training_labels = np.ndarray((TRAINING_AMOUNT,n))

test_data = np.ndarray((TEST_AMOUNT,68 * 2))
test_labels = np.ndarray((TEST_AMOUNT,n))

training_counter = 0
test_counter = 0



for i in range (n):
    name = input('name')
    classes.append(name)
    data = unpickle(STORE_LOCATION + name)
    
#     print(data)
    training_data[training_counter : training_counter + int(DATA_AMOUNT * 4 / 5),:] = data[: int(DATA_AMOUNT * 4 / 5), :, :].reshape(int(DATA_AMOUNT * 4 / 5), 68 * 2)
    test_data[test_counter : test_counter + int(DATA_AMOUNT * 1 / 5)] = data[int(DATA_AMOUNT * 4 / 5) :, :, :].reshape(int(DATA_AMOUNT * 1 / 5), 68 * 2)
   
    training_labels[training_counter :, :] = 0
    training_labels[training_counter : training_counter + int(DATA_AMOUNT * 4 / 5),i] = 1
    test_labels[test_counter :, :] = 0
    test_labels[test_counter : test_counter + int(DATA_AMOUNT * 1 / 5), i] = 1
    
    training_counter += int(DATA_AMOUNT * 4 / 5)
    test_counter += int(DATA_AMOUNT * 1 / 5)
training_data = torch.from_numpy(training_data)
training_labels = torch.from_numpy(training_labels)
# print(training_data[0])
# print(training_labels[0])
print(training_data.shape)
print(training_labels.shape)

print('done loading data')

amount of people:10
nameBone
nameChal
nameBook
nameGearng
nameNick
nameNut
nameOat
nameP'An
nameP'Dear
nameP'Ton
torch.Size([4000, 136])
torch.Size([4000, 10])
done loading data


In [66]:
# import torch
# def unpickle(file):
#     with open(file,'rb') as fo:
#         output = pickle.load(fo, encoding = 'ASCII')
#     return output
# n = int(input('amount of people:'))

# classes = list()

# TRAINING_AMOUNT = int(n * DATA_AMOUNT * 4 / 5)
# TEST_AMOUNT = int(n * DATA_AMOUNT * 1 / 5)

# training_data = np.ndarray((TRAINING_AMOUNT,68*2))
# training_labels = np.ndarray((TRAINING_AMOUNT,1))

# test_data = np.ndarray((TEST_AMOUNT,68 * 2))
# test_labels = np.ndarray((TEST_AMOUNT,1))

# training_counter = 0
# test_counter = 0



# for i in range (n):
#     name = input('name')
#     classes.append(name)
#     data = unpickle(STORE_LOCATION + name)
    
# #     print(data)
#     training_data[training_counter : training_counter + int(DATA_AMOUNT * 4 / 5),:] = data[: int(DATA_AMOUNT * 4 / 5), :, :].reshape(int(DATA_AMOUNT * 4 / 5), 68 * 2)
#     test_data[test_counter : test_counter + int(DATA_AMOUNT * 1 / 5)] = data[int(DATA_AMOUNT * 4 / 5) :, :, :].reshape(int(DATA_AMOUNT * 1 / 5), 68 * 2)
   
#     training_labels[training_counter : training_counter + int(DATA_AMOUNT * 4 / 5)] = i
#     test_labels[test_counter : test_counter + int(DATA_AMOUNT * 1 / 5)] = i
    
#     training_counter += int(DATA_AMOUNT * 4 / 5)
#     test_counter += int(DATA_AMOUNT * 1 / 5)
# training_data = torch.from_numpy(training_data)
# training_labels = torch.from_numpy(training_labels)
# # print(training_data[0])
# # print(training_labels[0])
# print(training_data.shape)
# print(training_labels.shape)

# # print('done loading data')

amount of people:4
nameBone
nameChal
nameTanawinBook
namethipok
torch.Size([1600, 136])
torch.Size([1600, 1])


In [4]:
print(training_labels[1199])


 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 10]



In [5]:
from torch.utils.data import Dataset, DataLoader
class landmarksData(Dataset):
    def __init__(self, data, labels, classes,transform=None, data_dir=None):
        self.data = data
        self.labels = labels
        self.transform=transform
        self.classes = classes
    def __getitem__(self,index):
        single_landmark_labels = self.labels[index]
        single_landmarks = self.data[index]
        return (single_landmarks,single_landmark_labels)
    def __len__(self):
        return len(self.data)
training_set = landmarksData(training_data, training_labels, classes)
test_set = landmarksData(test_data, test_labels, classes)
print(training_set.__len__())
print(test_set.__len__())
# print(training_set.__getitem__(3))

4000
1000


In [7]:
# print(training_data.type)
# print(training_labels.type)
# training_set = torch.utils.data.TensorDataset(training_data,training_labels)

In [6]:
trainloader = DataLoader(training_set, batch_size=4,shuffle=True)
dataiter = iter(trainloader)
landmarks, label = dataiter.next()
print("landmarks: ",landmarks[0])
print("label: ",label[0])
testloader = DataLoader(test_set, batch_size=4,shuffle=True)
testiter = iter(testloader)


landmarks:  
 0.0000
 0.1061
 0.0076
 0.2694
 0.0228
 0.4245
 0.0608
 0.5714
 0.1255
 0.7061
 0.2091
 0.8204
 0.3080
 0.9102
 0.4183
 0.9755
 0.5361
 0.9959
 0.6502
 0.9714
 0.7338
 0.8898
 0.8175
 0.8041
 0.8859
 0.6857
 0.9392
 0.5633
 0.9734
 0.4286
 0.9924
 0.2939
 0.9962
 0.1633
 0.1407
 0.0490
 0.2281
 0.0041
 0.3270
 0.0000
 0.4259
 0.0286
 0.5133
 0.0776
 0.6426
 0.0776
 0.7224
 0.0327
 0.8137
 0.0082
 0.8973
 0.0204
 0.9544
 0.0776
 0.5741
 0.1918
 0.5779
 0.2857
 0.5817
 0.3755
 0.5817
 0.4694
 0.4563
 0.5510
 0.5133
 0.5633
 0.5665
 0.5755
 0.6160
 0.5673
 0.6654
 0.5551
 0.2281
 0.1837
 0.2928
 0.1673
 0.3612
 0.1714
 0.4144
 0.2204
 0.3536
 0.2327
 0.2852
 0.2245
 0.6996
 0.2245
 0.7605
 0.1837
 0.8251
 0.1837
 0.8745
 0.2082
 0.8289
 0.2408
 0.7605
 0.2408
 0.3650
 0.7102
 0.4449
 0.6939
 0.5095
 0.6776
 0.5551
 0.6939
 0.6008
 0.6816
 0.6578
 0.7061
 0.7186
 0.7265
 0.6578
 0.7796
 0.6008
 0.8041
 0.5513
 0.8122
 0.5019
 0.8041
 0.4373
 0.7755
 0.3954
 0.7184
 0.5095
 0.

# Training

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
def foo(l, dtype=float):
    return map(dtype, l)

In [8]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(68 * 2, 68 * 2)
        self.fc2 = nn.Linear(68 * 2, 64)
        self.fc3 = nn.Linear(64, n)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# net.cuda()
print(net)

Net(
  (fc1): Linear(in_features=136, out_features=136)
  (fc2): Linear(in_features=136, out_features=64)
  (fc3): Linear(in_features=64, out_features=10)
)


In [19]:
import torch.optim as optim

criterion =  nn.MSELoss()
# optimizer = optim.SGD(net.parameters(), lr = 0.0001, momentum = 0.9)
optimizer = optim.SGD(net.parameters(), lr = 0.01)

In [ ]:
for epoch in range (10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs.float()), Variable(labels.float())
#         if(i == 1):
#             labelss=labels[:,0]
#             print("inputs: ", inputs)
#             print("outputs: ", outputs)
#             print("labels:", labelss)
        optimizer.zero_grad()
        outputs = net(inputs)
#         print('outputs:',outputs)
#         print(labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
        if(i % 100 == 99):
            print('[',epoch + 1,i + 1,']', ' loss: ',running_loss / 100)
            running_loss = 0.0
print('finished training')

[ 1 100 ]  loss:  0.027582438956014813
[ 1 200 ]  loss:  0.02814427500590682
[ 1 300 ]  loss:  0.026836001691408456
[ 1 400 ]  loss:  0.025255680081900208
[ 1 500 ]  loss:  0.027505785031244158
[ 1 600 ]  loss:  0.026155354068614543
[ 1 700 ]  loss:  0.026871621822938323
[ 1 800 ]  loss:  0.026790271000936627
[ 1 900 ]  loss:  0.026946685216389595
[ 1 1000 ]  loss:  0.024446669365279377
[ 2 100 ]  loss:  0.024247564151883127
[ 2 200 ]  loss:  0.026712465984746813
[ 2 300 ]  loss:  0.024684615111909808
[ 2 400 ]  loss:  0.026626556566916405
[ 2 500 ]  loss:  0.025314008013810962
[ 2 600 ]  loss:  0.02676909974310547
[ 2 700 ]  loss:  0.026071020355448126
[ 2 800 ]  loss:  0.026258331175195052
[ 2 900 ]  loss:  0.027338962368667126
[ 2 1000 ]  loss:  0.027675393261015414
[ 3 100 ]  loss:  0.02535041977185756
[ 3 200 ]  loss:  0.025008138874545692
[ 3 300 ]  loss:  0.02481859066989273
[ 3 400 ]  loss:  0.028119607996195554
[ 3 500 ]  loss:  0.025903044855222105
[ 3 600 ]  loss:  0.0266977

# Test accuracy

In [23]:
def outputToTensor(labels):
    # print(labels)
    # print(labels.numpy().astype(int))
    tmp = labels.numpy().astype(int)
    # print([ np.where(r==1)[0][0] for r in tmp ])
    tmp = [ np.where(r==1)[0][0] for r in tmp ]
    tmp = np.array(tmp)
    tmp = torch.from_numpy(tmp)
    # print(tmp)
    return tmp

In [37]:
dataiter  = iter(testloader)
landmarks,labels = dataiter.next()
print(labels)
tmp = outputToTensor(labels)
print('GroundTruth:' ,' '.join('%5s'% classes[int(tmp[j])] for j in range(4)))

outputs = net(Variable(landmarks.float()))
print('outputs:',outputs)
_,predicted = torch.max(outputs.data, 1)
print(predicted)
print('Predicted :',' '.join('%5s'% classes[int(predicted[j])] for j in range(4) ))


    0     0     0     0     0     0     0     1     0     0
    0     0     0     0     0     0     0     0     0     1
    0     0     0     0     0     0     0     1     0     0
    0     0     1     0     0     0     0     0     0     0
[torch.DoubleTensor of size 4x10]

GroundTruth:   Tan   Mum   Tan  Book
outputs: Variable containing:
-0.0790 -0.0196  0.3560  0.4597  0.0458 -0.0823  0.1725  0.2134 -0.0267 -0.0705
 0.1431  0.0125  0.0211  0.0604 -0.0033 -0.0352 -0.0003  0.0957  0.2255  0.4799
-0.0761 -0.0170  0.2782  0.3745  0.0427 -0.0733  0.1284  0.2982  0.0667 -0.0408
 0.5559  0.0796  0.0879  0.0069 -0.0152  0.1711  0.0655  0.0926 -0.0749 -0.0059
[torch.FloatTensor of size 4x10]


 3
 9
 3
 0
[torch.LongTensor of size 4]

Predicted : Thipok   Mum Thipok  Bone


In [25]:
correct = 0
total = 0
i = 0
for data in testloader:
    i += 1
    images,labels = data
    outputs = net(Variable(images.float()))
#     if( i < 5): 
#         print(labels)
#         print('output:',torch.max(outputs.data,1))
#     print(outputs)
    _,predicted = torch.max(outputs.data, 1)
#     print('predicted: ',predicted)
#     print('labels: ',labels)
    labels = outputToTensor(labels)
    total += labels.size(0)
    correct += (predicted == labels).sum()    
print('Accuracy :',(correct/total*100))

Accuracy : 70.39999999999999


In [26]:
print(net)

Net (
  (fc1): Linear (136 -> 136)
  (fc2): Linear (136 -> 64)
  (fc3): Linear (64 -> 10)
)


# Saving model

In [56]:
name = input('name: ')
torch.save(net.state_dict(), 'C:\\Users\\Bone\\Desktop\\faces\\models\\'+name)
# print('a')

name: secondTry


In [27]:
name = input('name: ')
pickle.dump(net,open('C:\\Users\\Bone\\Desktop\\faces\\models\\' + name, 'wb'))
print('saved')

name: 10 person
saved
